HÜSEYİN GÜLŞİN - 2020*******

Before looking for **CART** decision tree algorithm implementation here, you need to understand fundamentals of CART algorithm.(https://github.com/milaan9/Python_Decision_Tree_and_Random_Forest/blob/main/002_Decision_Tree_PlayGolf_CART.ipynb)[more detail about CART algorithm]

In [1]:
import pandas as pd

train_df = pd.read_csv('trainSet.csv')
test_df = pd.read_csv('testSet.csv')

We opened our csv data sets above. But there are some categorical variables in some colums. We can translate these categorial variables to numeric with Label Encder.(alternative: one-hot encoding).

In [2]:
from sklearn.preprocessing import LabelEncoder


Why **fit_transform** is used for the train while **transform** is for the test?

Because during preprocessing, train data learns and returns transformed data with fit_transform. To apply for test data same parameters that learned with fit_transform, you should use transform.


In [ ]:
le = LabelEncoder()

for col in ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']:
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

We need to create a Node to calculate all the leaves.
Nodes will decide left or right looking for the feature and its threshold. If low than the threshold then the left node otherwise right node.

In [ ]:
class Node:
    def __init__(self, data, target):
        self.left = None
        self.right = None
        self.data = data
        self.target = target
        self.feature = None
        self.threshold = None
        self.prediction = None

Gini Impurity is an important part of the during searching for leaves. It is a statistical measure to calculate homogeneity. The algorithm splits the dataset with the lowest index of Gini and threshold.

In [ ]:
def gini_impurity(groups, classes):
    n_instances = sum([len(group) for group in groups])
    gini = 0.0
    for group in groups:
        
        size = len(group)
        
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

We are listing left and right nodes.

In [ ]:
def test_split(index, value, data):
    left, right = list(), list()
    for row in data:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

Time to train data on Node. max_depth and min_samples parameters are required for Pre-pruning. This function works recursively on node.left and node.right.

In [ ]:
# Pre-pruning is the process of stopping the tree-building process early before it reaches maximum depth by setting some stopping criteria. In my code it is max_depth and min_samples parameters.

def get_split(data, max_depth, min_samples, depth=1):
    class_values = list(set(row[-1] for row in data))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(data[0])-1):
        for row in data:
            groups = test_split(index, row[index], data)
            
            gini = gini_impurity(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    node = Node(data, class_values)
    if b_score == 0:
        node.prediction = max(set([row[-1] for row in data]), key=[row[-1] for row in data].count)
        return node
    if depth >= max_depth or len(data) <= min_samples:
        node.prediction = max(set([row[-1] for row in data]), key=[row[-1] for row in data].count)
        return node
    node.feature = b_index
    node.threshold = b_value
    node.left = get_split(b_groups[0], max_depth, min_samples, depth+1)
    node.right = get_split(b_groups[1], max_depth, min_samples, depth+1)
    return node



root = get_split(train_df.values, 5, 5)  # Maximum depth: 5, Minimum number of samples: 5

After training, we need to see our results. In order to evaluate tp, tn, fp, and fn, you should use the following steps; we are sending the last form of the node to predict function. Then we are storing the predictions that come from test data. Finally, evaluate with prediction and actual values. 

In [ ]:
def predict(node, row):
    if node.prediction is not None:
        return node.prediction
    if row[node.feature] < node.threshold:
        return predict(node.left, row)
    else:
        return predict(node.right, row)

In [ ]:
def test_tree(root, test_data):
    predictions = []
    for i in range(len(test_data)):
        result = predict(root, test_data.iloc[i])
        predictions.append(result)
    return predictions

In [ ]:
def evaluate(predictions, actual):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(predictions)):   
        if predictions[i] == actual[i]:
            if predictions[i] == "good":
                tp += 1
            else:
                tn += 1
        else:
            if predictions[i] == "good":
                fp += 1
            else:
                fn += 1
    accuracy = (tp + tn) / len(predictions)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    return accuracy, tpr, tnr, tp, tn


For instance, test_predictions is equal to stored predictions that come from test_tree function, train_actual is equal to train data which comes from the class column and lastly sending this data to evaluate function to obtain the following results: 
train_accuracy, train_tpr, train_tnr, train_tp, train_tn

In [ ]:
train_predictions = test_tree(root, train_df)
train_actual = train_df['class']
train_accuracy, train_tpr, train_tnr, train_tp, train_tn = evaluate(train_predictions, train_actual)

print('Eğitim (Train) sonucu:')
print('Accuracy:', train_accuracy)
print('True Positive Rate:', train_tpr)
print('True Negative Rate:', train_tnr)
print('True Positive Count:', train_tp)
print('True Negative Count:', train_tn)

test_predictions = test_tree(root, test_df)
test_actual = test_df['class']
test_accuracy, test_tpr, test_tnr, test_tp, test_tn = evaluate(test_predictions, test_actual)

print('\nSınama (Test) sonucu:')
print('Accuracy:', test_accuracy)
print('True Positive Rate:', test_tpr)
print('True Negative Rate:', test_tnr)
print('True Positive Count:', test_tp)
print('True Negative Count:', test_tn)